In [ ]:
# as always, starting spark session for map reduce tasks
# spark for this large dataset of 380000 & 8000 truncated row number's
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Powerlifting MapReduce Task") \
    .getOrCreate()


In [ ]:
# uploading the dataset from my computer to colab
from google.colab import files

uploaded = files.upload()
uploaded = files.upload()

uploaded  # just to see the files names uploaded here

print("files uploaded:", list(uploaded.keys()))


In [ ]:
# reading the csv files
# i use pandas (filter, group, simulations) because mapreduce here is easier to simulate in python (mapper + reducer)
meets_path = "meets.csv"
opl_path = "openpowerlifting.csv"

meets_df = pd.read_csv(meets_path)
opl_df = pd.read_csv(opl_path)

print("meets rows:", len(meets_df))
print("openpowerlifting rows:", len(opl_df))

# checking columns to avoid silly mistakes with names
print("meets columns:", list(meets_df.columns))
print("opl columns:", list(opl_df.columns))

meets_df.head(3)
meets_df.head(3)


LITTLE CLEANNING ~~~~~~~~~~

In [ ]:
# basic cleaning only for columns we need for the 2 task objective yey
# i only keep the needed cols to avoid big memory usage (poor computer)

meets_small = meets_df[["MeetID", "MeetName", "MeetCountry", "Date"]].copy()
opl_small = opl_df[["MeetID", "Name", "TotalKg"]].copy()

# removing rows with missing meetid or name because they break the groupings
meets_small = meets_small.dropna(subset=["MeetID", "MeetName", "MeetCountry", "Date"])
opl_small = opl_small.dropna(subset=["MeetID", "Name"])

# converting meetid to int (sometimes it reads as float)
meets_small["MeetID"] = meets_small["MeetID"].astype(int)
opl_small["MeetID"] = opl_small["MeetID"].astype(int)

# totalkg can be missing or text, so i convert and keep numeric
opl_small["TotalKg"] = pd.to_numeric(opl_small["TotalKg"], errors="coerce")

print("after cleaning -> meets:", len(meets_small), "| opl:", len(opl_small))
opl_small.head(3)


2.1 COUNT PARTICIPATES x Meet ID ~~~~~~~~

In [ ]:
# mapreduce idea done would be
# mapper: (MeetID kew competition value-> 1 per assigned participant)
# for every row (one row = one lifter in that meet)
# reducer: sum all 1s per MeetID => total Nº PARTICIPANTS x ID

def mapper_participants(opl_rows):
    mapped = [] #empty list
    for meet_id in opl_rows["MeetID"]:
        mapped.append((meet_id, 1)) # pair generated
    return mapped

def reducer_sum_counts(mapped_pairs):
    reduced = {} # dictionary
    for k, v in mapped_pairs: #k meet id, v is 1
        reduced[k] = reduced.get(k, 0) + v #meet id exist => actual value, if not, returns 0.+1
    return reduced

mapped_participants = mapper_participants(opl_small)
participants_count = reducer_sum_counts(mapped_participants)

print("example of reduced result (first 5 meetids):")
print(list(participants_count.items())[:5])


In [ ]:
# turning the reducer output into a dataframe for printing nicely
# list => TABLE
participants_df = pd.DataFrame(list(participants_count.items()), columns=["MeetID", "Participants"])
# order from more to less nº participant competitions (more important ones)
participants_df = participants_df.sort_values("Participants", ascending=False)

participants_df.head(10)


2.2 COMPETITIONS BEING LISTED ~~~~~~

In [ ]:
# here year is needed for analysis (not all date)
# meets has Date like "2016-10-29" so i take the first 4 chars
# to string date, pick 4 chars (year)
meets_small["Year"] = meets_small["Date"].astype(str).str[:4]

# sometimes date can be weird, so i keep only valid numeric year
meets_small = meets_small[meets_small["Year"].str.isnumeric()] # true if numbers
meets_small["Year"] = meets_small["Year"].astype(int) # to int (sort, group, analysis later)

meets_small.head(3)


In [ ]:
# i already have participants per MeetID from prevous task 2 point 1
# now i "join" with meets info to get name/country/year
# 1 row x competition
meets_info = meets_small.drop_duplicates(subset=["MeetID"])[["MeetID", "MeetName", "MeetCountry", "Year"]]
# unify tables meet id as key that have meets info (nor participants? => NaN)
competitions_df = meets_info.merge(participants_df, on="MeetID", how="left")

# if some meetid has no participants, it will be NaN -> i set to 0
competitions_df["Participants"] = competitions_df["Participants"].fillna(0).astype(int)

# sorting to see the biggest events
competitions_df = competitions_df.sort_values(["Participants"], ascending=False)

competitions_df.head(15)


In [ ]:
# saving the final competitions table because it is a requested output basically
competitions_df.to_csv("task2_competitions_list.csv", index=False)

print("saved -> task2_competitions_list.csv")


2.3 TOTAL WEIGHT LIFTED x PARTICIPANT, ALL COMPETITIONS taken into account ~~~~~~

In [ ]:
# here the mpreducer
# mapper pairs: (Name per person , TotalKg) for each row (one result of a lifter)
# reducer dicc: sum all TotalKg per Name across all meets
#
def mapper_total_by_lifter(opl_rows):
    mapped = []
    for _, row in opl_rows.iterrows():
        name = row["Name"]
        total = row["TotalKg"]

        # if total is missing i skip it (otherwise it becomes nan sums)
        if pd.notna(total):
            mapped.append((name, float(total))) # name? is float to good sum?
    return mapped

def reducer_sum_totals(mapped_pairs):
    reduced = {}
    for k, v in mapped_pairs: #name, total kg
        reduced[k] = reduced.get(k, 0.0) + v # name not exixst, starts at 0.0 + total kg
    return reduced

mapped_totals = mapper_total_by_lifter(opl_small)
total_by_lifter = reducer_sum_totals(mapped_totals)

print("example (first 5 lifters):")
print(list(total_by_lifter.items())[:5])


In [ ]:
# converting reducer output to dataframe
# items => pairs of dicc total_by_lifter, pd.df to create table
lifters_df = pd.DataFrame(list(total_by_lifter.items()), columns=["Name", "TotalKg_AllCompetitions"])
lifters_df = lifters_df.sort_values("TotalKg_AllCompetitions", ascending=False)
# people compiting lots of times or years
lifters_df.head(15) #visual


In [ ]:
# saving the output of course (as every task in 2)
lifters_df.to_csv("task2_total_by_lifter.csv", index=False)

print("saved -> task2_total_by_lifter.csv")
